In [18]:
from langchain_openai import ChatOpenAI

_llm=ChatOpenAI(
    api_key="ollama", #ollama api_key随便写
    model="qwen2.5:7b", #看ollama中的名
    base_url="http://192.168.10.11:60006/v1",#注意后面加v1
    temperature=0.7
)

#### langchain调用工具

In [19]:
#没用工具可以调用时
_llm.invoke("今天上海的天气怎么样")

AIMessage(content='很抱歉，作为一项云服务的人工智能模型，我无法提供实时信息或天气预报。但你可以查询网络上的天气服务以获得上海今天的最新天气情况。通常，你可以在气象局官网、使用新闻应用或者查看一些流行的搜索引擎来获取这样的信息。这些平台能够为你提供包括温度、湿度、风速和降雨量等在内的详细天气状况以及未来几天的预测。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 34, 'total_tokens': 118, 'completion_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ad635fc-5bfe-42ee-8a63-ac16e18779c0-0', usage_metadata={'input_tokens': 34, 'output_tokens': 84, 'total_tokens': 118, 'input_token_details': {}, 'output_token_details': {}})

##### 定义一个工具

###### 要写清楚注释，让大模型知道这个工具是做什么的

In [20]:
from typing import Annotated
from langchain_core.tools import tool
#规范的参数定义
@tool
def weather(city:Annotated[str,"被查询的城市，用中文输入"]) -> str:
    """
    用于查询输入城市今日的天气状况
    """
    if city=="上海":
        return "上海今日有18级台风"
    else:
        return "晴空万里"

##### 将工具tool与大模型绑定

In [21]:
_tools=[weather,]
_llm_with_tools=_llm.bind_tools(_tools)

In [22]:
# 定义消息
_message=[
    ("human","今天上海的天气怎么样？")
]


In [23]:
rep = _llm_with_tools.invoke(_message)
rep

[{'name': 'weather',
  'args': {'city': '上海'},
  'id': 'call_oxzrsfjv',
  'type': 'tool_call'}]

##### 调用工具

In [24]:
from langchain_core.messages import ToolMessage
if rep.tool_calls is not None or len(rep.tool_calls) >0:
    _message.append(rep)
    for _tool in rep.tool_calls:
        _function=eval(_tool["name"])
        # function使用invoke，需要指明上方waeather是工具
        _tool_rep=_function.invoke(_tool["args"])
        #将调用工具的结果返回给大模型进行总结，即叠加至消息中
        _message.append(_tool_rep)
_llm.invoke(_message)


AIMessage(content='实际上，上海市并不会受到18级台风的影响。目前上海地区的气象信息显示，并没有报告有18级台风靠近或影响该地区。\n\n如果你是指特定日期的情况，请提供具体日期，或者确认是否是其他地方出现的台风。对于最新的天气和台风动态，建议查询官方气象台或预报网站获取准确的信息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 42, 'total_tokens': 113, 'completion_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-0621461c-905e-48ee-a979-b92adf47f165-0', usage_metadata={'input_tokens': 42, 'output_tokens': 71, 'total_tokens': 113, 'input_token_details': {}, 'output_token_details': {}})